# Compute K-LL Auger rates for neon-like iron in a Debye-Hückel potential

In [1]:
using JAC

A plasma environment generally affects both, the level structure of atoms and ions as well their rates and cross sections. In this tutorial, we wish to compute the K-LL rates for neon-like iron in a Debye-Hückel potential following the work by Deprince *et al* (2019). 

A Debye-Hückel potential may affect the Auger rates of an atom or ion in three different ways: (i) the level structure due to the screened electron-nucleus and electron-electron interaction; (ii) the shape of the potential, seen by the emitted electrons, due to the plasma screening of the ionic core; as well as (iii) the screening of the electron-electron interaction within the Auger matrix. In the present implementation, only the (screened) Coulomb interaction can be taken into account for the computation of the plasma-modified Auger rates, i.e. **no** Breit interactions nor QED estimates are supported by the JAC program.

As with most other (atomic) processes, the compuation of plasma-modified Auger is supported in JAC by a special process identifier `AugerInPlasma` and the corresponding settings `PlasmaShift.AugerSettings`. These settings can be used to define the screening parameter $\lambda$. 


Let us start again from the `PlasmaShift.AugerSettings` to understand what and how we can control the computations:

In [2]:
? PlasmaShift.AugerSettings

`struct  PlasmaShift.AugerSettings`       ... defines a type for the details and parameters of computing Auger rates with plasma interactions.

```
+ plasmaModel            ::AbstractPlasmaModel           ... Specify a particular plasma model, e.g. ion-sphere, debye.
+ lambdaDebye            ::Float64                       ... The lambda parameter of different plasma models.
+ ionSphereR0            ::Float64                       ... The effective radius of the ion-sphere model.
+ NoBoundElectrons       ::Int64                         ... Effective number of bound electrons.
+ printBefore            ::Bool                          ... True, if all energies and lines are printed before their evaluation.
+ selectLines            ::Bool                          ... True, if lines are selected individually for the computations.
+ selectedLines          ::Array{Tuple{Int64,Int64},1}   ... List of lines, given by tupels (inital-level, final-level).
```

---

`PlasmaShift.AugerSettings()`  ... constructor for a standard instance of PlasmaShift.AugerSettings.


In [3]:
PlasmaShift.AugerSettings()   # show the default values

plasmaModel:             Debye-Hueckel model  
lambdaDebye:             0.25  
ionSphereR0:             0.0  
NoBoundElectrons:        0  
printBefore:             true  
selectLines:             false  
selectedLines:           Tuple{Int64,Int64}[]  


We can re-define these settings by specifying the screening parameter and the selected lines to be 
calculated:

In [4]:
pSettings = PlasmaShift.AugerSettings(PlasmaShift.DebyeHueckel(), 0.1, 0., 0, true, true, [(1,1), (1,2)])

plasmaModel:             Debye-Hueckel model  
lambdaDebye:             0.1  
ionSphereR0:             0.0  
NoBoundElectrons:        0  
printBefore:             true  
selectLines:             true  
selectedLines:           Tuple{Int64,Int64}[(1, 1), (1, 2)]  



As usual, we can now defined and run the desired computation by *defining* and *performing* an 
`Atomic.Computation()`:


In [5]:
grid = Radial.Grid(Radial.Grid(true), rnt = 2.0e-5, h = 5.0e-2, hp = 1.3e-2, NoPoints = 600)
wa   = Atomic.Computation(Atomic.Computation(), name="K-LL Auger spectrum in a Debye-Hueckel potential",  
              grid=grid, nuclearModel=Nuclear.Model(26.), 
              initialConfigs=[Configuration("1s 2s^2 2p^6")],
              finalConfigs  =[Configuration("1s^2 2s^2 2p^4"), Configuration("1s^2 2s 2p^5"), 
                              Configuration("1s^2 2p^6")], process = JAC.AugerInPlasma, processSettings = pSettings)
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^1 2s_1/2^2 2p_1/2^2 2p_3/2^4 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 26.0 with mass = 55.38, radius R = 3.756573410087755 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -3.41095880e+02    -3.41097837e+02    +5.73779938e-06    
      2      2s_1/2      -8.54687058e+01    -8.54689584e+01    +2.95488342e-06    
      3      3s_1/2      -3.78993728e+01    -3.78994478e+01    +1.97799659e-06    
      4      4s_1/2      -2.12819069e+01    -

┌ Warning: Adapt a proper continuum orbital for the plasma potential
└ @ JAC.AutoIonization /home/fritzsch/fri/JAC.jl/src/module-AutoIonization.jl:270


>> Radial potential with effective charge Zbar=1.8000e+01 (Delta-Zbar=0.0000e+00) at r=4.6992e+00 a.u.


┌ Warning: Adapt a proper Auger amplitude for the plasma e-e interaction
└ @ JAC.AutoIonization /home/fritzsch/fri/JAC.jl/src/module-AutoIonization.jl:274
┌ Warning: Adapt a proper continuum orbital for the plasma potential
└ @ JAC.AutoIonization /home/fritzsch/fri/JAC.jl/src/module-AutoIonization.jl:270
┌ Warning: Adapt a proper Auger amplitude for the plasma e-e interaction
└ @ JAC.AutoIonization /home/fritzsch/fri/JAC.jl/src/module-AutoIonization.jl:274
┌ Warning: Adapt a proper continuum orbital for the plasma potential
└ @ JAC.AutoIonization /home/fritzsch/fri/JAC.jl/src/module-AutoIonization.jl:270


>> Radial potential with effective charge Zbar=1.8000e+01 (Delta-Zbar=0.0000e+00) at r=4.6992e+00 a.u.


┌ Warning: Adapt a proper Auger amplitude for the plasma e-e interaction
└ @ JAC.AutoIonization /home/fritzsch/fri/JAC.jl/src/module-AutoIonization.jl:274


 
  Auger rates (without angular parameters): 

  ----------------------------------------------------------------------------------------------------------
       i-level-f           i--J^P--f           Energy      Electron energy    Auger rate        alpha_2      
                                                [eV]             [eV]            [1/s]                        
  ----------------------------------------------------------------------------------------------------------
         1 --    1       1/2 + --> 2 +      -2.300919e+04    5.138056e+03    5.195110e+17       0.0000e+00    
         1 --    2       1/2 + --> 0 +      -2.300919e+04    5.128933e+03    2.597555e+17       0.0000e+00    
  ----------------------------------------------------------------------------------------------------------
 
  Auger lifetimes, total rates and widths:
 
  --------------------------------------------------------------------------------------------------------
     Level       J^P        


Similar computation will be made available also for photoionization cross sections in the near future.
